In [1]:
from __future__ import print_function

import numpy as np
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

import argparse
import math
import os
import shutil
import time
import random
from functools import partial

from tensorboardX import SummaryWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.optim.lr_scheduler import _LRScheduler

from attacker import NoOpAttacker, PGDAttacker
import net_cifar
import net
from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig
from utils.fastaug.fastaug import FastAugmentation
from utils.fastaug.augmentations import Lighting
from utils.misc import show_image_row
from utils.label_maps import CLASS_DICT
from models.stem_helper import *

In [ ]:
resume = "/ws/external/checkpoints/"
load = "/ws/external/checkpoints/"
gpu_id = "0"
num_classes = 10
mixbn = False
mixstem = False
lr_schedule = "cos"

In [2]:
def to_status(m, status):
    if hasattr(m, 'batch_type'):
        m.batch_type = status


to_clean_status = partial(to_status, status='clean')
to_adv_status = partial(to_status, status='adv')
to_mix_status = partial(to_status, status='mix')

# Models
default_model_names = sorted(name for name in net_cifar.__dict__ if
                             name.islower() and not name.startswith('__') and callable(
                                 net_cifar.__dict__[name]) and not name.startswith("to_") and not name.startswith(
                                 "partial"))

model_names = default_model_names


In [3]:
# Use CUDA
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
use_cuda = torch.cuda.is_available()
# Random seed
manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if use_cuda:
    torch.cuda.manual_seed_all(manualSeed)
best_acc = 0  # best test accuracy


NameError: name 'args' is not defined